In [10]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [11]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [12]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [13]:
# 2. Create a temporary view of the DataFrame.
#df = spark.read.csv(url, header=True, inferSchema=True)
#df.createOrReplaceTempView("home_sales_revised")
#df.createOrReplaceTempView("home_sales")
local_directory = "/Resources/"

# Define the file name for the downloaded CSV file
file_name = "home_sales_revised.csv"

# Combine the local directory and file name to create the full local file path
local_file_path = local_directory + file_name

# Download the CSV file from the URL and save it locally (you may need to install the 'requests' library)
import requests
response = requests.get(url)
with open(local_file_path, "wb") as file:
    file.write(response.content)

# Read the locally saved CSV file into a DataFrame
df = spark.read.csv(local_file_path, header=True, inferSchema=True)

# Create a temporary view
df.createOrReplaceTempView("home_sales")

In [15]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
average_price_query = """
SELECT YEAR(date) AS Year, 
       ROUND(AVG(price), 2) AS AveragePrice
FROM home_sales
WHERE bedrooms = 4
GROUP BY Year
ORDER BY Year
"""

# Execute the query and show the result
result = spark.sql(average_price_query)
result.show()

+----+------------+
|Year|AveragePrice|
+----+------------+
|2019|    300263.7|
|2020|   298353.78|
|2021|   301819.44|
|2022|   296363.88|
+----+------------+



In [17]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
average_price_query = """
SELECT date_built AS Year, 
       ROUND(AVG(price), 2) AS AveragePrice
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY Year
"""

# Execute the query and show the result
result = spark.sql(average_price_query)
result.show()

+----+------------+
|Year|AveragePrice|
+----+------------+
|2010|   292859.62|
|2011|   291117.47|
|2012|   293683.19|
|2013|   295962.27|
|2014|   290852.27|
|2015|    288770.3|
|2016|   290555.07|
|2017|   292676.79|
+----+------------+



In [18]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
average_price_query = """
SELECT date_built AS Year, 
       ROUND(AVG(price), 2) AS AveragePrice
FROM home_sales
WHERE bedrooms = 3 
      AND bathrooms = 3 
      AND floors = 2 
      AND sqft_living >= 2000
GROUP BY date_built
ORDER BY Year
"""

# Execute the query and show the result
result = spark.sql(average_price_query)
result.show()


+----+------------+
|Year|AveragePrice|
+----+------------+
|2010|   285010.22|
|2011|   276553.81|
|2012|   307539.97|
|2013|   303676.79|
|2014|   298264.72|
|2015|   297609.97|
|2016|    293965.1|
|2017|   280317.58|
+----+------------+



In [19]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
import time

# Record the start time
start_time = time.time()

# Your query execution code here
average_price_query = """
SELECT `view`, ROUND(AVG(price), 2) AS AveragePrice
FROM home_sales
WHERE price >= 350000
GROUP BY `view`
"""

result = spark.sql(average_price_query)
result.show()

# Record the end time
end_time = time.time()

# Calculate and print the elapsed time
elapsed_time = end_time - start_time
print("--- %s seconds ---" % elapsed_time)

+----+------------+
|view|AveragePrice|
+----+------------+
|  31|   399856.95|
|  85|  1056336.74|
|  65|   736679.93|
|  53|    755214.8|
|  78|  1080649.37|
|  34|   401419.75|
|  81|  1053472.79|
|  28|   402124.62|
|  76|  1058802.78|
|  26|   401506.97|
|  27|   399537.66|
|  44|   400598.05|
|  12|   401501.32|
|  91|  1137372.73|
|  22|   402022.68|
|  93|  1026006.06|
|  47|    398447.5|
|   1|   401044.25|
|  52|   733780.26|
|  13|   398917.98|
+----+------------+
only showing top 20 rows

--- 0.5116996765136719 seconds ---


In [20]:
# 7. Cache the the temporary table home_sales.
df.cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [21]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
import time

# Record the start time for the uncached query
start_time_uncached = time.time()

# Run the query without caching
uncached_query = """
SELECT `view`, ROUND(AVG(price), 2) AS AveragePrice
FROM home_sales
WHERE price >= 350000
GROUP BY `view`
"""

uncached_result = spark.sql(uncached_query)
uncached_result.show()

# Record the end time for the uncached query
end_time_uncached = time.time()

# Calculate and print the elapsed time for the uncached query
elapsed_time_uncached = end_time_uncached - start_time_uncached
print("Uncached Runtime: %s seconds" % elapsed_time_uncached)

# Now, cache the DataFrame and run the query again

# Cache the temporary table home_sales
df.cache()

# Record the start time for the cached query
start_time_cached = time.time()

# Run the query with caching
cached_result = spark.sql(uncached_query)
cached_result.show()

# Record the end time for the cached query
end_time_cached = time.time()

# Calculate and print the elapsed time for the cached query
elapsed_time_cached = end_time_cached - start_time_cached
print("Cached Runtime: %s seconds" % elapsed_time_cached)

+----+------------+
|view|AveragePrice|
+----+------------+
|  31|   399856.95|
|  85|  1056336.74|
|  65|   736679.93|
|  53|    755214.8|
|  78|  1080649.37|
|  34|   401419.75|
|  81|  1053472.79|
|  28|   402124.62|
|  76|  1058802.78|
|  26|   401506.97|
|  27|   399537.66|
|  44|   400598.05|
|  12|   401501.32|
|  91|  1137372.73|
|  22|   402022.68|
|  93|  1026006.06|
|  47|    398447.5|
|   1|   401044.25|
|  52|   733780.26|
|  13|   398917.98|
+----+------------+
only showing top 20 rows

Uncached Runtime: 1.9501638412475586 seconds
+----+------------+
|view|AveragePrice|
+----+------------+
|  31|   399856.95|
|  85|  1056336.74|
|  65|   736679.93|
|  53|    755214.8|
|  78|  1080649.37|
|  34|   401419.75|
|  81|  1053472.79|
|  28|   402124.62|
|  76|  1058802.78|
|  26|   401506.97|
|  27|   399537.66|
|  44|   400598.05|
|  12|   401501.32|
|  91|  1137372.73|
|  22|   402022.68|
|  93|  1026006.06|
|  47|    398447.5|
|   1|   401044.25|
|  52|   733780.26|
|  13|   

In [25]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("/Desktop/Parquet/")

Py4JJavaError: An error occurred while calling o105.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.j

In [ ]:
# 11. Read the formatted parquet data.


In [ ]:
# 12. Create a temporary table for the parquet data.


In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.


In [ ]:
# 15. Check if the home_sales is no longer cached

